In [1]:
import numpy as np
import pandas as pd

In [2]:
rag_features=np.load("rag_features.npy")

In [3]:
cnn_features=pd.read_csv("cnn_features_large.csv")

In [4]:
cnn_features.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,22,23,24,25,26,27,28,29,30,31
0,0,-2.576777,-3.945932,4.803016,3.189036,-2.080729,-1.822992,-1.415842,1.251550,-1.122755,...,2.900688,0.836991,-0.162772,0.327203,-3.030703,4.946615,-2.560668,-0.702272,0.138903,2.183795
1,1,-1.273053,-2.931379,6.150136,3.694960,-4.095263,-1.475067,-2.136545,1.437015,-2.896256,...,2.814346,-0.179391,-1.063998,-0.518706,-4.148804,3.832036,-3.757436,2.780955,0.713002,4.008554
2,2,-5.547634,-7.335995,4.485065,3.809859,-1.156559,-0.141361,-3.035493,0.800622,-0.905164,...,4.025711,4.258149,-1.408270,-1.450558,-3.353894,4.266500,-3.481836,-5.016182,0.472003,1.710162
3,3,-2.219563,-3.196903,5.448789,3.788954,-1.905776,-0.782672,-2.127305,-0.263901,-1.591551,...,2.539538,0.982034,-0.976380,-0.613500,-2.164868,3.118608,-2.345061,-0.255565,1.632033,3.254106
4,4,-6.854525,-7.804424,5.984237,3.427894,1.028706,-1.682487,-1.388808,0.809004,3.089412,...,0.865692,3.909653,0.915837,1.075717,0.670368,6.161496,0.199500,-10.325129,0.261942,-0.152172


In [5]:
cnn_features.drop(["Unnamed: 0"],axis=1,inplace=True)

In [6]:
cnn_features=cnn_features.values

In [7]:
rag_features.shape

(10015, 12100)

In [8]:
cnn_features.shape

(10015, 32)

In [9]:
total_features=np.hstack((rag_features,cnn_features))

In [10]:
total_features.shape

(10015, 12132)

In [11]:
def get_Y_orig(labels_loc):
    #output_list=list()
    Y_df = pd.read_csv(labels_loc)
    Y_df=Y_df.iloc[:,1:]
#     for index,row in Y_df.iterrows():
#         if row["MEL"]==1:
#             output_list.append(0)
#         elif row["NV"]==1:
#             output_list.append(1)
#         elif row["BCC"]==1:
#             output_list.append(2)
#         elif row["AKIEC"]==1:
#             output_list.append(3)
#         elif row["BKL"]==1:
#             output_list.append(4)
#         elif row["DF"]==1:
#             output_list.append(5)
#         elif row["VASC"]==1:
#             output_list.append(6)
    return Y_df

In [12]:
labels_loc = 'ISIC2018_Task3_Training_GroundTruth/ISIC2018_Task3_Training_GroundTruth.csv'

Y_orig = get_Y_orig(labels_loc)

In [13]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [14]:
model = Sequential()
model.add(Dense(12, input_dim=12132, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(7, activation='softmax'))

In [15]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
from sklearn.model_selection import train_test_split
train_X,val_X,train_Y,val_Y= train_test_split(total_features, Y_orig, test_size=0.2, shuffle= True)

In [17]:
model.fit(train_X, train_Y, epochs=50, batch_size=64,shuffle=True)

Epoch 1/50
8012/8012 [==============================] - 2s 277us/step - loss: 1.1817 - accuracy: 0.6030
Epoch 2/50
8012/8012 [==============================] - 2s 260us/step - loss: 0.4374 - accuracy: 0.8628
Epoch 3/50
8012/8012 [==============================] - 2s 293us/step - loss: 0.2995 - accuracy: 0.9217
Epoch 4/50
8012/8012 [==============================] - 2s 229us/step - loss: 0.2255 - accuracy: 0.9443
Epoch 5/50
8012/8012 [==============================] - 2s 238us/step - loss: 0.1777 - accuracy: 0.9551
Epoch 6/50
8012/8012 [==============================] - 2s 233us/step - loss: 0.1422 - accuracy: 0.9608
Epoch 7/50
8012/8012 [==============================] - 2s 271us/step - loss: 0.1150 - accuracy: 0.9680
Epoch 8/50
8012/8012 [==============================] - 2s 290us/step - loss: 0.0928 - accuracy: 0.9749
Epoch 9/50
8012/8012 [==============================] - 2s 295us/step - loss: 0.0738 - accuracy: 0.9799
Epoch 10/50
8012/8012 [==============================] - 2s 211u

In [18]:
_, accuracy = model.evaluate(val_X, val_Y)
print('Accuracy: %.2f' % (accuracy*100))

2003/2003 [==============================] - 1s 332us/step
Accuracy: 94.16


In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 12)                145596    
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 63        
Total params: 145,763
Trainable params: 145,763
Non-trainable params: 0
_________________________________________________________________


In [22]:
predictions=model.predict_classes(val_X)

In [23]:
predictions=predictions.tolist()

In [24]:
output_list=list()
for index,row in val_Y.iterrows():
    if row["MEL"]==1:
        output_list.append(0)
    elif row["NV"]==1:
        output_list.append(1)
    elif row["BCC"]==1:
        output_list.append(2)
    elif row["AKIEC"]==1:
        output_list.append(3)
    elif row["BKL"]==1:
        output_list.append(4)
    elif row["DF"]==1:
        output_list.append(5)
    elif row["VASC"]==1:
        output_list.append(6)

In [25]:
from sklearn.metrics import classification_report

In [26]:
classification_report(output_list,predictions,output_dict=True)

{'0': {'precision': 0.8947368421052632,
  'recall': 0.8018867924528302,
  'f1-score': 0.845771144278607,
  'support': 212},
 '1': {'precision': 0.9542857142857143,
  'recall': 0.9723435225618632,
  'f1-score': 0.9632299927901947,
  'support': 1374},
 '2': {'precision': 0.96, 'recall': 0.96, 'f1-score': 0.96, 'support': 100},
 '3': {'precision': 1.0,
  'recall': 0.9830508474576272,
  'f1-score': 0.9914529914529915,
  'support': 59},
 '4': {'precision': 0.8708133971291866,
  'recall': 0.883495145631068,
  'f1-score': 0.8771084337349399,
  'support': 206},
 '5': {'precision': 0.9259259259259259,
  'recall': 0.8620689655172413,
  'f1-score': 0.8928571428571429,
  'support': 29},
 '6': {'precision': 1.0,
  'recall': 0.8260869565217391,
  'f1-score': 0.9047619047619047,
  'support': 23},
 'accuracy': 0.9415876185721418,
 'macro avg': {'precision': 0.9436802684922985,
  'recall': 0.8984188900203385,
  'f1-score': 0.9193116585536831,
  'support': 2003},
 'weighted avg': {'precision': 0.9411443